# Import Libraries - 
    This analysis is done  with  using data from investing.com 

In [1]:
import investpy
import pandas as pd 
import numpy as np 
import matplotlib as  plt  
# Import date class from datetime module
import  datetime 
# Returns the current local date
today = datetime.date.today()
print("Today date is: ", today)
pd.set_option("display.max_columns", 999)

Today date is:  2021-04-04


## Input timeline for analysis 

In [14]:
def selectTheTimeline( days):
    fromdate =  today -  datetime.timedelta(days = days)
    todate   =  datetime.datetime.strptime(str(today), "%Y-%m-%d").strftime("%d/%m/%Y")
    fromdate =  datetime.datetime.strptime(str(fromdate), "%Y-%m-%d").strftime("%d/%m/%Y")
    #print ('The Range of Analysis is -  ', fromdate , ' & ', todate )
    return (todate, fromdate)

## Index for Analysis 
       1.Get Data 
       2.Design features - this is as per best knowledge of my understanding of market trends

In [35]:
def index_calculations (indices_name):
    # Get the data into DataFrame 
    df = pd.DataFrame (investpy.get_index_historical_data(index=indices_name
                                        , country='india'
                                        , from_date= fromdate
                                        , to_date= todate))
    #df["Id"] = df.in`dex + 1    
    #df['Index_Name'] = indices_name
    #print ('Columns Name \n', df.columns)
    data =  pd.merge(df, df.shift(1), on="Date", how="left",suffixes = ('_Cur','_Prev')) 
    #print ('Columns Name \n', data.columns)
    #print ('Data  \n', data.head(5))
    # Design the rules 
    data['Index_Name'] = indices_name
    # Featuring Engineering
    data['Open_Close']  =  data['Close_Cur'] - data['Open_Cur']
    data['Day_Range']   =  data['High_Cur'] - data['Low_Cur']
    # previous day broken  DB Stands for Day Broken 
    data['PrevDB_High'] =  np.where(data["High_Cur"]  > data["High_Prev"], 1, 0)
    data['PrevDB_Low']  =  np.where(data["Low_Cur"]  > data["Low_Prev"], 1, 0)
    # Gap opening 
    data['Gap']  =  data["Open_Cur"]  -  data["Close_Prev"] 
    # Central Pivot Point (P) = (High + Low + Close) / 3
    data['CPR']  =  ( data["High_Prev"] + data["Low_Prev"] + data["Close_Prev"])/3 
    #Get moving Average 
    data['MA_200D_Close_Cur'] =  data['Close_Cur'].rolling(window=201).mean()
    data['MA_100D_Close_Cur'] =  data['Close_Cur'].rolling(window=101).mean()
    data['MA_50D_Close_Cur']  =  data['Close_Cur'].rolling(window=51).mean()
    data['MA_20D_Close_Cur']  =  data['Close_Cur'].rolling(window=21).mean()
    data['MA_10D_Close_Cur']  =  data['Close_Cur'].rolling(window=11).mean()
    # 52 weeks high 
    data['52_weeks_high'] =  data['Close_Cur'].rolling(window=365).max()
    # 52 weeks low 
    data['52_weeks_low'] =  data['Close_Cur'].rolling(window=365).min()
    #  Get the cross over right 
    data['MA_Diff_200_100'] =  data['MA_200D_Close_Cur'] - data['MA_100D_Close_Cur']
    data['MA_Diff_100_50'] =   data['MA_100D_Close_Cur'] - data['MA_50D_Close_Cur']
    data['MA_Diff_50_20']  =   data['MA_50D_Close_Cur'] - data['MA_20D_Close_Cur']
    data['MA_Diff_20_10']  =   data['MA_20D_Close_Cur'] - data['MA_10D_Close_Cur']
    #indices_data.append(data, ignore_index = True)
    #print ('Final Data  \n', data.head(5))
    return data

In [36]:
#indices_data =  []
print ( 'enter the time line into years ')
timeline =  int( input())
timeline = timeline*365             
todate, fromdate = selectTheTimeline(timeline)
print ('The Range of Analysis is -  ', fromdate , ' & ', todate )

indices_name = ['Nifty Bank']#,'Nifty 50']
itr = 1 
for i in indices_name:
    print (i)
    data2=index_calculations(i)
    print (itr)
    if (itr == 1):
        #print ('Final Data before append  \n', data.head(5))
        indices_data = data2.copy()
        #print (' Indices Data  \n', indices_data.head(5))
    else :
        indices_data = pd.concat([indices_data, data2])
    #print ('Final Data  \n', indices_data.head(5))
    itr = itr+1 

enter the time line into years 
2
The Range of Analysis is -   05/04/2019  &  04/04/2021
Nifty Bank
1


In [37]:
data2.sort_values('Date',ascending=False).head(5)

,Open_Cur,High_Cur,Low_Cur,Close_Cur,Volume_Cur,Currency_Cur,Open_Prev,High_Prev,Low_Prev,Close_Prev,Volume_Prev,Currency_Prev,Index_Name,Open_Close,Day_Range,PrevDB_High,PrevDB_Low,Gap,CPR,MA_200D_Close_Cur,MA_100D_Close_Cur,MA_50D_Close_Cur,MA_20D_Close_Cur,MA_10D_Close_Cur,52_weeks_high,52_weeks_low,MA_Diff_200_100,MA_Diff_100_50,MA_Diff_50_20,MA_Diff_20_10
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-01,33588.00,33949.15,33208.75,33858.00,312718,INR,33764.80,33773.10,33151.80,33303.90,370017.0,INR,Nifty Bank,270.00,740.40,1,1,284.1,33409.600000,27436.063184,32317.081683,34482.974510,34574.871429,33699.127273,37306.25,16917.65,-4881.018499,-2165.892827,-91.896919,875.744156
2021-03-31,33764.80,33773.10,33151.80,33303.90,370017,INR,33615.90,33990.80,33564.55,33875.10,374228.0,INR,Nifty Bank,-460.90,621.30,0,0,-110.3,33810.150000,27366.684577,32236.139109,34451.382353,34643.347619,33785.181818,37306.25,16917.65,-4869.454532,-2215.243244,-191.965266,858.165801
2021-03-30,33615.90,33990.80,33564.55,33875.10,374228,INR,33605.05,33610.65,33166.10,33318.20,291665.0,INR,Nifty Bank,259.20,426.25,1,1,297.7,33364.983333,27303.752488,32152.857921,34436.006863,34714.761905,33955.968182,37306.25,16917.65,-4849.105433,-2283.148942,-278.755042,758.793723
2021-03-26,33605.05,33610.65,33166.10,33318.20,291665,INR,33420.75,33550.15,32415.25,33006.45,486186.0,INR,Nifty Bank,-286.85,444.55,1,1,598.6,32990.616667,27237.334826,32054.103465,34410.507843,34842.090476,34103.381818,37306.25,16917.65,-4816.768639,-2356.404378,-431.582633,738.708658
2021-03-25,33420.75,33550.15,32415.25,33006.45,486186,INR,33894.70,33970.65,33203.40,33293.25,345484.0,INR,Nifty Bank,-414.30,1134.90,0,0,127.5,33489.100000,27176.548259,31962.754950,34391.307843,34991.333333,34341.572727,37306.25,16917.65,-4786.206692,-2428.552893,-600.025490,649.760606


In [ ]:
indices_data.groupby(['Index_Name']).size()

In [ ]:
data.sort_values('Date',ascending=False).head(5)

In [ ]:
df.shift(1).sort_values('Date',ascending=False).head(5)

In [ ]:
df=investpy.get_stocks(country="india")
df_indices= pd.DataFrame(investpy.get_indices(country="india"))
df_indices['type'] =  df_indices['name'].str[:3] 
df_indices.groupby(['type']).count()
df_indices[df_indices['type']== 'Nif']
indices_name =df_indices['name'][df_indices['type']== 'Nif']


In [ ]:
df_indices[df_indices['name'] == 'NSE%']

In [ ]:
investpy.get_index_historical_data(index='ibex 35', country='spain', from_date='01/01/2018', to_date='01/01/2019')

In [28]:
df_stock_list= pd.DataFrame(investpy.get_stocks(country="india"))
df_stock_list['type'] =  df_stock_list['name'].str[:3] 
print ('the data for stockes \n ', df_stock_list.head(10) )
df_stock_list.groupby(['type']).count()

the data for stockes 
    country                  name                 full_name          isin  \
0   india  Aditya Birla Capital  Aditya Birla Capital Ltd  INE674K01013   
1   india               Hubtown               Hubtown Ltd  INE703H01016   
2   india           3i Infotech           3i Infotech Ltd  INE748C01020   
3   india              3M India             3M India Ltd.  INE470A01017   
4   india             ABB India             ABB India Ltd  INE117A01022   
5   india                   ACC                   ACC Ltd  INE012A01025   
6   india      Aarti Industries      Aarti Industries Ltd  INE769A01020   
7   india         Aban Offshore        Aban Offshore Ltd.  INE421A01028   
8   india          ABG Shipyard         ABG Shipyard Ltd.  INE067H01016   
9   india     Adani Enterprises     Adani Enterprises Ltd  INE423A01024   

  currency symbol type  
0      INR   ADTB  BSE  
1      INR   HUBT  BSE  
2      INR   TIIN  S&P  
3      INR   TMIN  BSE  
4      INR    ABB  BSE  


,country,name,full_name,isin,currency,symbol
type,,,,,,
BNY,12,12,12,12,12,12
BSE,36,36,36,36,36,36
DJ,8,8,8,8,8,8
FTS,2,2,2,2,2,2
Ind,2,2,2,2,2,2
MCI,3,3,3,3,3,3
MCX,6,6,6,6,6,6
MSC,3,3,3,3,3,3
NIF,3,3,3,3,3,3


In [34]:
help (investpy.get_stock_information)

Help on function get_stock_information in module investpy.stocks:

get_stock_information(stock, country, as_json=False)
    This function retrieves fundamental financial information from the specified stock. The retrieved 
    information from the stock can be valuable as it is additional information that can be used combined 
    with OHLC values, so to determine financial insights from the company which holds the specified stock.
    
    Args:
        stock (:obj:`str`): symbol of the stock to retrieve its information from.
        country (:obj:`country`): name of the country from where the stock is from.
        as_json (:obj:`bool`, optional):
            optional argument to determine the format of the output data (:obj:`dict` or :obj:`json`).
    
    Returns:
        :obj:`pandas.DataFrame` or :obj:`dict`- stock_information:
            The resulting :obj:`pandas.DataFrame` contains the information fields retrieved from Investing.com
            from the specified stock ; it c